# Get Stock Data
Using yfinance api to get stock data in the form of Open, High, Low, Close price points of stock candlesticks

In [8]:
import pandas as pd
from pandas_datareader import data as web
import yfinance as yf
yf.pdr_override()
import numpy as np

STOCK = "ICICIBANK.NS"
START_DATE = "2022-05-05"
END_DATE = "2022-07-07"
STOCK_PATTERN = "TRIANGLE"

df = web.get_data_yahoo(STOCK, start=START_DATE, end=END_DATE)
df['Id'] = np.arange(1, len(df)+1)
df.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Id
Date,,,,,,,
2022-05-05,730.799988,743.000000,726.500000,728.799988,724.452576,9121547,1
2022-05-06,716.250000,723.000000,710.299988,719.250000,714.959534,12455025,2
2022-05-09,705.299988,720.000000,704.450012,710.349976,706.112610,13055889,3
2022-05-10,705.000000,720.000000,705.000000,711.299988,707.056946,12797419,4
2022-05-11,718.000000,719.799988,707.150024,714.200012,709.939697,13516852,5
2022-05-12,707.549988,710.700012,693.299988,695.799988,691.649414,16685166,6
2022-05-13,702.400024,702.400024,675.000000,677.349976,673.309448,15717043,7
2022-05-16,676.700012,690.950012,675.750000,683.000000,678.925781,9025623,8
2022-05-17,686.099976,712.000000,685.150024,710.000000,705.764709,12765052,9


# Get Pivot points
Pivot points are points representing a particular candlestick which has High price point greater than NUM_BEFORE candles or NUM_AFTER candles. Same for the Low price points as well.

In [9]:
NUM_BEFORE = 3
NUM_AFTER = 3

def pivotId(df, candle, num_before, num_after):
    if candle-num_before < 0 or candle+num_after >= len(df):
        return 0
    
    pivotIdLow=1
    pivotIdHigh=1
    for i in range(candle-num_before, candle+num_after):
        if(df.Low[candle]>df.Low[i]):
            pivotIdLow=0
        if(df.High[candle]<df.High[i]):
            pivotIdHigh=0
    if pivotIdLow and pivotIdHigh:
        return 3
    elif pivotIdLow:
        return 1
    elif pivotIdHigh:
        return 2
    else:
        return 0
    
df['Pivot'] = df.apply(lambda row: pivotId(df, int(row.Id)-1, NUM_BEFORE, NUM_AFTER), axis=1)
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot
Date,,,,,,,,
2022-06-23,690.200012,706.000000,687.000000,699.150024,694.979492,14960412,36,0
2022-06-24,710.900024,715.250000,706.549988,713.450012,709.194153,10856736,37,0
2022-06-27,728.849976,728.849976,716.099976,717.349976,713.070862,8285225,38,2
2022-06-28,711.049988,717.299988,707.000000,710.299988,706.062927,8339931,39,0
2022-06-29,704.950012,706.700012,700.950012,702.750000,698.557983,9570557,40,0
2022-06-30,702.750000,711.900024,701.200012,707.200012,702.981445,11970404,41,0
2022-07-01,703.450012,706.750000,694.099976,703.900024,699.701111,9884899,42,1
2022-07-04,704.849976,721.099976,704.849976,720.099976,715.804443,8303732,43,0
2022-07-05,723.049988,732.150024,718.099976,719.450012,715.158386,11460367,44,0


# Fix position of Pivot points
The position of pivot points is fixed just above or below a particular candle

In [10]:
def pointPosition(x):
    if x['Pivot']==1:
        return x['Low']-(0.01*df.High.max())
    elif x['Pivot']==2:
        return x['High']+(0.01*df.High.max())
    else:
        return np.nan

df['PointPosition'] = df.apply(lambda row: pointPosition(row), axis=1)
pd.set_option("display.max_rows", None, "display.max_columns", None)
display(df)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot,PointPosition
Date,,,,,,,,,
2022-05-05,730.799988,743.000000,726.500000,728.799988,724.452576,9121547,1,0,NaN
2022-05-06,716.250000,723.000000,710.299988,719.250000,714.959534,12455025,2,0,NaN
2022-05-09,705.299988,720.000000,704.450012,710.349976,706.112610,13055889,3,0,NaN
2022-05-10,705.000000,720.000000,705.000000,711.299988,707.056946,12797419,4,0,NaN
2022-05-11,718.000000,719.799988,707.150024,714.200012,709.939697,13516852,5,0,NaN
2022-05-12,707.549988,710.700012,693.299988,695.799988,691.649414,16685166,6,0,NaN
2022-05-13,702.400024,702.400024,675.000000,677.349976,673.309448,15717043,7,1,667.430000
2022-05-16,676.700012,690.950012,675.750000,683.000000,678.925781,9025623,8,0,NaN
2022-05-17,686.099976,712.000000,685.150024,710.000000,705.764709,12765052,9,0,NaN


# Plot Pivot points

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['PointPosition'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Draw Pattern
The algorithm focuses on recent triangle pattern as a triangle pattern with future possibility of breakout is useful for a trader to make informed decision.

In [12]:
from scipy.stats import linregress

RECENT_HIGH_PIVOT_POINT = 0
RECENT_LOW_PIVOT_POINT = 0

FEASIBLE_HIGH_PIVOT_POINTS = np.array([])
FEASIBLE_LOW_PIVOT_POINTS = np.array([])
FEASIBLE_HIGH = np.array([])
FEASIBLE_LOW = np.array([])

for i in reversed(range(len(df))):
    if df.Pivot[i] == 2 and RECENT_HIGH_PIVOT_POINT == 0:
        RECENT_HIGH_PIVOT_POINT = df.Id[i]
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, df.High[i])
    if df.Pivot[i] == 1 and RECENT_LOW_PIVOT_POINT == 0:
        RECENT_LOW_PIVOT_POINT = df.Id[i]
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, df.Low[i])
    if RECENT_HIGH_PIVOT_POINT != 0 and RECENT_LOW_PIVOT_POINT != 0:
        break

# Triangle Pattern
if STOCK_PATTERN == "TRIANGLE":
    try:
        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, RECENT_HIGH_PIVOT_POINT)
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, RECENT_LOW_PIVOT_POINT)

        dfHigh=df[df['Pivot']==2]
        dfLow=df[df['Pivot']==1]

        MAX_HIGH_PIVOT_POINT = dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'Id'].iloc[0]
        MIN_LOW_PIVOT_POINT = dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Id'].iloc[0]

        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, MAX_HIGH_PIVOT_POINT)
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'High'])
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, MIN_LOW_PIVOT_POINT)
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Low'])

        dfHigh = dfHigh[dfHigh['Id'] > MAX_HIGH_PIVOT_POINT]
        dfLow = dfLow[dfLow['Id'] > MIN_LOW_PIVOT_POINT]

        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'Id'].iloc[0])
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'High'])
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Id'].iloc[0])
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Low']) 

        slmin, intercmin, rmin, pmin, semin = linregress(FEASIBLE_LOW_PIVOT_POINTS, FEASIBLE_LOW)
        slmax, intercmax, rmax, pmax, semax = linregress(FEASIBLE_HIGH_PIVOT_POINTS, FEASIBLE_HIGH)

        # To show stock data after END DATE to visualize stock trend after pattern
        df1 = web.get_data_yahoo(STOCK, start=END_DATE)
        df1['Id'] = np.arange(df.Id.max(), len(df1)+df.Id.max())

        df = pd.concat([df, df1])
        
        fig = go.Figure(data=[go.Candlestick(x=df.Id,
                        open=df['Open'],
                        high=df['High'],
                        low=df['Low'],
                        close=df['Close'])])

        fig.add_scatter(x=df.Id, y=df['PointPosition'], mode="markers",
                        marker=dict(size=5, color="MediumPurple"),
                        name="Pivot")

        fig.add_trace(go.Scatter(x=FEASIBLE_LOW_PIVOT_POINTS, y=slmin*FEASIBLE_LOW_PIVOT_POINTS + intercmin, mode='lines', line=dict(color="Blue"), name='min slope'))
        fig.add_trace(go.Scatter(x=FEASIBLE_HIGH_PIVOT_POINTS, y=slmax*FEASIBLE_HIGH_PIVOT_POINTS + intercmax, mode='lines', line=dict(color="Blue"), name='max slope'))

        fig.update_layout(
            font_size=20,
        )
        fig.show()

    except:
        print("No pattern detected")
        exit()



[*********************100%***********************]  1 of 1 completed


In [13]:
from scipy import stats
A,B,C,D,E = stats.linregress(FEASIBLE_HIGH_PIVOT_POINTS, FEASIBLE_HIGH)


print(C*C)
print(D)
print(C)

1.0
9.003163161571059e-11
-1.0
